In [1]:
import MDAnalysis as mda
import nglview as nv
import pathlib

from MDAnalysis.analysis.rms import rmsd
from rdkit import Chem
from rdkit.Chem import AllChem

from openff.toolkit.topology import Molecule

In [2]:
env_193 = "fb-193-tk-010-oe-2022"
env_195 = "fb-195-tk-013-oe-2022-interchange-replace-cache"

In [4]:
here = pathlib.Path(".")
sdf = list(here.glob("targets/*/*.sdf"))[0]
system = sdf.stem
batch = sdf.parent.stem

In [5]:
batch

'opt-geo-batch-48'

In [6]:
system

'18437974-17'

In [8]:
u1 = mda.Universe(
    (
        f"{env_193}/"
        "rep1/optimize.tmp/"
        f"{batch}/iter_0000/"
        f"{system}_mmopt.xyz"
    )
)
nv.show_mdanalysis(u1)

NGLWidget()

In [9]:
u2 = mda.Universe(
    (
        f"{env_195}/"
        "rep1/optimize.tmp/"
        f"{batch}/iter_0000/"
        f"{system}_mmopt.xyz"
    )
)
nv.show_mdanalysis(u2)

NGLWidget()

In [10]:
mol_qm = Molecule.from_file(
    f"targets/{batch}/{system}.sdf", "SDF",
    allow_undefined_stereo=True
)
rdmol_qm = mol_qm.to_rdkit()
u_qm = mda.Universe(rdmol_qm)
nv.show_mdanalysis(u_qm)

NGLWidget()

In [17]:
u1_to_u2 = rmsd(u1.atoms.positions, u2.atoms.positions)
u1_to_qm = rmsd(u1.atoms.positions, u_qm.atoms.positions)
u2_to_qm = rmsd(u2.atoms.positions, u_qm.atoms.positions)

print(f"   1.9.3 to QM: {u1_to_qm:.2f}")
print(f"   1.9.5 to QM: {u2_to_qm:.2f}")
print(f"1.9.3 to 1.9.5: {u1_to_u2:.2f}")

   1.9.3 to QM: 0.90
   1.9.5 to QM: 0.75
1.9.3 to 1.9.5: 0.55


In [18]:
rdmol1 = u1.atoms.convert_to("RDKIT")
rdmol2 = u2.atoms.convert_to("RDKIT")

r1_to_r2 = AllChem.GetBestRMS(rdmol1, rdmol2)
r1_to_qm = AllChem.GetBestRMS(rdmol1, rdmol_qm)
r2_to_qm = AllChem.GetBestRMS(rdmol2, rdmol_qm)

print(f"   1.9.3 to QM: {r1_to_qm:.2f}")
print(f"   1.9.5 to QM: {r2_to_qm:.2f}")
print(f"1.9.3 to 1.9.5: {r1_to_r2:.2f}")

   1.9.3 to QM: 0.69
   1.9.5 to QM: 0.69
1.9.3 to 1.9.5: 0.00


/lila/home/lilywang/micromamba/envs/fit-virtual-sites-tk010-py39/lib/python3.9/site-packages/MDAnalysis/converters/RDKit.py:472: UserWarning: No `bonds` attribute in this AtomGroup. Guessing bonds based on atoms coordinates
  warnings.warn(


In [15]:
mol1 = Molecule.from_rdkit(rdmol1, allow_undefined_stereo=True)
mol2 = Molecule.from_rdkit(rdmol2, allow_undefined_stereo=True)

In [16]:
mol1.to_smiles(mapped=True)

'[H:20][c:2]1[c:4]([c:8]([c:5]([c:3]([c:7]1[C:1]#[N:15])[H:21])[H:23])/[C:12](=[N:18]/[N:19]([H:32])[c:11]2[n:16][c:9]([c:6]([c:10]([n:17]2)[C:14]([H:29])([H:30])[H:31])[H:24])[C:13]([H:26])([H:27])[H:28])/[H:25])[H:22]'

In [17]:
mol2.to_smiles(mapped=True)

'[H:20][c:2]1[c:4]([c:8]([c:5]([c:3]([c:7]1[C:1]#[N:15])[H:21])[H:23])/[C:12](=[N:18]/[N:19]([H:32])[c:11]2[n:16][c:9]([c:6]([c:10]([n:17]2)[C:14]([H:29])([H:30])[H:31])[H:24])[C:13]([H:26])([H:27])[H:28])/[H:25])[H:22]'

In [18]:
mol1.to_smiles(mapped=True) == mol2.to_smiles(mapped=True)

True